Step 1: Initial protocell structure optimisation
Structure optimisation using BeGeN2 as an example, prepare POSCAR, INCAR, POTCAR

####INCAR ####
Global Parameters
ISTART   =  0              (Read existing wavefunction; if there)
ISPIN    =  0              (Non-Spin polarised DFT)
# ICHARG =  11             (Non-self-consistent: GGA/LDA band structures)
LREAL    = .FALSE.         (Projection operators: automatic)
ENCUT    =  700            (Cut-off energy for plane wave basis set, in eV)
PREC     =  A              (Precision level)
LWAVE    = .FALSE.         (Write WAVECAR or not)
LCHARG   = .FALSE.         (Write CHGCAR or not)
ADDGRID  = .TRUE.          (Increase grid; helps GGA convergence)

#SYMPREC = 1E-6 

Lattice Relaxation
NELM     = 200
POTIM    = 0.1
NPAR    = 2

NSW      =  300            (number of ionic steps)
ISMEAR   =  0              (gaussian smearing method )
SIGMA    =  0.05           (please check the width of the smearing)
IBRION   =  1              (Algorithm: 0-MD; 1-Quasi-New; 2-CG)
ISIF     =  3              (optimize atomic coordinates and lattice parameters) 
EDIFF    = 1E-7           
EDIFFG   = -1e-3          (Ionic convergence; eV/AA)

LVDW = .T.
IVDW = 12

Step 2: Construct the training set
After we get the high precision protocells, we expand the cells by phononpy to get the supercells then we get the configuration of the training set by manual perturbation, AIMD, and active learning, we give the code for manual perturbation below.

In [ ]:
#!/usr/bin/env python 
# coding: utf-8

from dpdata import LabeledSystem,MultiSystems
import os 
import dpdata
import shutil

def GetFileFromThisRootDir(dir,ext = None):
    allfiles = []
    needExtFilter = (ext != None)
    for root,dirs,files in os.walk(dir):
        for filespath in files:
            #filepath = os.path.join(root, filespath)
            filepath = os.path.join( filespath)
            extension = os.path.splitext(filepath)[1][1:]
            if needExtFilter and extension in ext:
                allfiles.append(filepath)
            elif not needExtFilter:
                allfiles.append(filepath)
    return allfiles
    
def del_folder(filepath):
    """
    删除某一目录下的所有文件或文件夹
    :param filepath: 路径
    :return:
    """
    del_list = os.listdir(filepath)
    for f in del_list:
        file_path = os.path.join(filepath, f)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
            
# 转换文件格式和编码方式
structure_name_list = GetFileFromThisRootDir( "./input/")
if os.path.exists('./output_structure')==False:
    os.mkdir('./output_structure')
else:
    del_folder('./output_structure')
    
print("每个构型扰动生成多少帧构型! ")
num_frame = int(input(""))
print("晶胞扰动的幅度(%)! ")
cell_pert_fraction = float(input(""))
print("原子距离扰动的幅度,单位埃! ")
atom_pert_distance = float(input(""))

for structure_name in structure_name_list:
    print(structure_name)
    perturbed_system = dpdata.System('./input/'+structure_name,fmt="vasp/poscar").perturb(pert_num=num_frame, 
        cell_pert_fraction=cell_pert_fraction, 
        atom_pert_distance=atom_pert_distance, 
        atom_pert_style='normal')#normal-正态分布 uniform-限定长度均匀散点 const-常数
    for i in range(num_frame):
        perturbed_system.to('vasp/poscar','./output_structure/' +structure_name+"-"+str(i+1),frame_idx=i)

After obtaining the configurations of the training set, we perform a single-point energy calculation to obtain the energy and force of each configuration.

#####manual perturbation #####
SYSTEM=BeGeN2
ISTART=0
ICHARG=2
ENCUT=700
ISMEAR=0
SIGMA=0.1
EDIFF=1E-7
NELMIN=5
EDIFFG=-1E-4
ISIF=3
IBRION=-1
POTIM=0.1
NSW=0
PREC=A
IALGO=38
LWAVE=F
LCHARG=F
LREAL=F
ADDGRID=T
NPAR=2
LVDW = .T.
IVDW = 12

Part 3: Training Machine Learning Potentials
After we get the training set, we fit the machine learning potential by GPUMD with the following control file

type         	3 Be Ge N
version 		4
model_type  	0
neuron			50			   # 默认 30
cutoff    		6  4           # 默认 8 4
n_max			10 8
basis_size		10 8			   # 默认 8 8 
l_max			4 2 1		   # 默认 4 2 0
lambda_e		1			   
lambda_f		1			   # 默认 1
lambda_v		0.1
lambda_shear	1
force_delta     0		
batch			1000 		   # 默认 1000
population      50 			   # 默认 50
generation    	400000
